In [2]:
# Set dependencies and import libraries
from splinter import Browser
from bs4 import BeautifulSoup, Comment
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

import time
import pandas as pd
import datetime
import pymongo
from time import sleep
from random import randint
import re

In [6]:
# Set some variables
dlr_name = ""
date = datetime.date.today()
scraped_url = "https://www.carsguide.com.au/buy-a-car/all-new-and-used/all-states/all-locations/all-bodytypes/all-makes/?searchTypes=USED%20Dealer%20ad,USED%20Private%20ad&distanceFromMe=Perth,WA"

In [7]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(scraped_url)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\chris\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [5]:
# Define clean-up functions
# def clean_name():
#     return dealer_name.replace('\n','')
    # dealer_name = dealer_name.replace('Message','')
    # dealer_name = dealer_name.strip()



2021-10-08


In [8]:
#  Define collection
cars = [] # To store list of dictionaries
pages = 5279 # Set the number of pages to scroll through
date = datetime.date.today()
date = date.strftime('%Y-%m-%d')

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['carsguide_used_cars_australia']
col_cars = db[f'scraped_cars_{date}']
col_sub_stats = db['suburb_stats']

# Set for loop to iterate through all pages
for x in range(pages):

    # Set HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain listing data
    results = soup.find_all('div', class_='carListing--text')
    # Retrieve all elements that contain dealer name
    dealer_data = soup.find_all('div', class_='cgForm')

    # Iterate through each listing
    for result, dealer in zip(results, dealer_data):
        
        try:
            year_make_model = result.find('h5', class_='carListing--textHeading').text
        except:
            year_make_model = "NULL"
        try:
            km = result.find('li', class_='icon-gauge').text
        except:
            km = "NULL"
        try:
            trans = result.find('li', class_='icon-gears').text
        except:
            trans = "NULL"
        try:
            price = result.find('h6', class_='carListing--textPrice').text
        except:
            price = "NULL"
        try:
            comment = result.find('div', class_='carListing--comments').text
        except:
            comment = "NULL"
        try:
            listing_type = result.find('li', class_='sellerCircle').text
        except:
            listing_type = "NULL"
        try:
            suburb = result.find('li', class_='icon-pin').text
        except:
            suburb = "NULL" 
        try:
            dealer_name = dealer.find('h1').text
        except:
            dealer_name = "NULL"

        # Clean up data
        dealer_name = dealer_name.replace('\n','')
        dealer_name = dealer_name.replace('Message','')
        dealer_name = dealer_name.strip()
        km = km.replace(',','')
        km = km.replace('km','')
        price = price.replace(',','')
        price = price.replace('$','')
        comment = comment.replace('\n','')
        source = scraped_url
        year = year_make_model[:4]
        make = year_make_model.split(' ')[1]
        model = year_make_model.split(' ')[2:]
        state = suburb.split(', ')[1]
        suburb = suburb.split(', ')[0]

        car = {"Dealer Name":dealer_name,\
             "Year-make-model":year_make_model,\
                 'KMs':km,\
                     'Tansmission':trans,\
                         'Price':price,\
                             'Comments':comment,\
                                    'Source URL': source,\
                                        'Year': year,\
                                            'Make': make,\
                                                'Model': model,\
                                                    'State Abrev': state,\
                                                        'Suburb': suburb,\
                                                            'Listing Type':listing_type,\
                                                                'Date scraped':date,\
                                                                    'Page Number':x}
        print('-------------------')
        print(dealer_name)
        print(year_make_model)
        print(km)
        print(trans)
        print(price)
        print(comment)
        print(listing_type)
        print(suburb)
        print('-------------------')

        # Update MongoDB
        col_cars.insert_one(car)
        #Append list of dictionaries
        cars.append(car)

        print('-------------------')
        print(f'page: {x} of {pages}')
        print('-------------------')

    try:
        nextpage = browser.find_by_css('.listing-pagination-next').first #Do I need to change this to last?
        nextpage.click()

    except:
        print("Scraping Complete")

# Quit browser
browser.quit()

-------------------
City Car Gallery Pty Ltd
2013 Renault Master MWB
96132 
Manual
26990
2013 Renault Master X62 MWB White 6 Speed Manual Van
USED Dealer ad
Bankstown
-------------------
-------------------
page: 0 of 5279
-------------------
-------------------
Peninsula Motor Group Used
2014 Hyundai Elantra Trophy
45141 
Automatic
16990
*****Covid 19 Update***** We are open for online sales offering contactless click and collect ...
USED Dealer ad
Peninsula Motor Group UsedBankstown
-------------------
-------------------
page: 0 of 5279
-------------------
-------------------
Sydney Wide European Autos
2009 Volkswagen Caddy Maxi
148570 
Manual
19990
2009 VOLKSWAGEN CADDY ,5 SPEED Manual, 1.9litre turbo diesel , REFERGATION/FREEZER VAN, goes down ...
USED Dealer ad
Condell Park
-------------------
-------------------
page: 0 of 5279
-------------------
-------------------

2011 Hyundai I30 SX
139145 
Automatic
6450
It's been well looked after and serviced regularly. It is excellent i

In [6]:
# Create DF
df = pd.DataFrame(cars)
# Create statistics by suburb
suburb_counts = df['Suburb'].value_counts().to_frame('Count').reset_index()
suburb_counts.head(30)
# suburb_avg_price = df['Price'].apply(pd.DataFrame.describe)
suburb_counts

,index,Count
0,Victoria Park,1147
1,Minchinbury,1131
2,Wangara,1007
3,Bentley,891
4,Berwick,689
...,...,...
1912,Fern Bay,1
1913,Chirnside Park,1
1914,Potts Point,1
1915,Doolandella,1
